### Group Assignment
### Group : 18

## Process of Preparing Wheat prices dataset by adding external factors data from other data sources

Following steps to do preprocessing and data cleaning procedures to refine the combined dataset, resulting in a consolidated dataset tailored for forecasting models.



In [74]:
# importing necessary libraries
import pandas as pd
import random

### Step1: Loading food price dataset for India from WFP website
https://data.humdata.org/dataset/wfp-food-prices-for-india

In [75]:
# Food Price India Dataset url
csv_url = "https://data.humdata.org/dataset/dc663585-4b6f-46ae-a6d6-b2f3e4ea32b5/resource/3b1ff071-6b01-4998-aa62-2f3efb5d4888/download/wfp_food_prices_ind.csv"
wheat_df =[]
try:
    df = pd.read_csv(csv_url,header=0, skiprows=[1])
    df['commodity_id'] = pd.to_numeric(df['commodity_id'], errors='coerce')
    print("Data loaded successfully from URL.")
except Exception as e:
    print(f"Error loading data from URL: {e}")
print("Total Dataset rows and cols")
df.shape

Data loaded successfully from URL.
Total Dataset rows and cols


(148420, 16)

### Step2: Data clean up and filtering "Wheat" data.

In [76]:
# Data cleanup and rename the column names

# Removing unnamed columns if any
wheat_df = df.drop(columns=[col for col in df.columns if "Unnamed" in col])

# Replace '#date' with 'date' to rename the column
wheat_df = wheat_df.rename(columns={'#date': 'date'})
wheat_df['date'] = pd.to_datetime(wheat_df['date'], errors='coerce')

# Filtering commodity with Wheat and year < 2023
wheat_df = wheat_df[(wheat_df['commodity_id']==84) & (wheat_df['date'] < '2023-01-01')]

#Rename admin1 and admin2 columns for better clarity
wheat_df = wheat_df.rename(columns={'admin1': 'state', 'admin2': 'region'})

print("Total Dataset rows and cols")
print(wheat_df.shape)
print("Dataset Info")
print(wheat_df.info())

Total Dataset rows and cols
(8519, 16)
Dataset Info
<class 'pandas.core.frame.DataFrame'>
Index: 8519 entries, 1 to 131346
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          8519 non-null   datetime64[ns]
 1   state         8495 non-null   object        
 2   region        8495 non-null   object        
 3   market        8519 non-null   object        
 4   market_id     8519 non-null   int64         
 5   latitude      8495 non-null   float64       
 6   longitude     8495 non-null   float64       
 7   category      8519 non-null   object        
 8   commodity     8519 non-null   object        
 9   commodity_id  8519 non-null   int64         
 10  unit          8519 non-null   object        
 11  priceflag     8519 non-null   object        
 12  pricetype     8519 non-null   object        
 13  currency      8519 non-null   object        
 14  price         8519 non-null   float64  

### Step3: Loading external dataset that effects food price dataset

Source links for external data are following:

1. Diesel data from
https://www.indexmundi.com/commodities/?commodity=diesel&months=240&currency=inr&commodity=wheat

2. Rainfall in subdividions of India from
https://www.data.gov.in/catalog/rainfall-india?page=2%0Bhttps%3A%2F%2Fwww.data.gov.in%2Fcatalog%2Frainfall-india


3. Consumer Price Index (CPI) from
 https://www.macrotrends.net/global-metrics/countries/IND/india/inflation-rate-cpi

4. Minimum Support Price (MSP) for Foodgrains from
https://www.rbi.org.in/scripts/PublicationsView.aspx?id=22498

All these files are downloaded manually and merged to the food price dataset after data cleanup & preprocessing.

In [77]:
# Load external datasets
cpi_df = pd.read_excel("/content/CPI.xlsx")
diesel_df = pd.read_excel("/content/diesel-240.xlsx")
msp_df = pd.read_excel("/content/MSP.xlsx")
rainfall_df = pd.read_csv("/content/Rainfall Sub_Division.csv")

# Display the heads to verify structure
{
    "wheat": wheat_df.head(),
    "CPI": cpi_df.head(),
    "Diesel": diesel_df.head(),
    "MSP": msp_df.head(),
    "Rainfall": rainfall_df.head()
}


{'wheat':          date           state           region     market  market_id  \
 1  1994-01-15           Delhi            Delhi      Delhi        934   
 9  1994-01-15       Karnataka  Bangalore Urban  Bengaluru        926   
 12 1994-01-15  Madhya Pradesh           Bhopal     Bhopal        928   
 23 1994-02-15           Delhi            Delhi      Delhi        934   
 27 1994-02-15         Gujarat        Ahmadabad  Ahmedabad        923   
 
     latitude  longitude            category commodity  commodity_id unit  \
 1      28.67      77.22  cereals and tubers     Wheat            84   KG   
 9      12.96      77.58  cereals and tubers     Wheat            84   KG   
 12     23.27      77.40  cereals and tubers     Wheat            84   KG   
 23     28.67      77.22  cereals and tubers     Wheat            84   KG   
 27     23.03      72.62  cereals and tubers     Wheat            84   KG   
 
    priceflag pricetype currency  price  usdprice  
 1     actual    Retail      INR   

### Step4: Clean and load the external files (CPI, Diesel Prices and Rainfall)

In [78]:
# Load the datasets, handling potential extra headers
def load_and_clean_csv(file_path):
    """Loads a CSV, handling potential extra header rows."""

    # Attempt to read with header=0 (default)
    try:
        df = pd.read_csv(file_path, header=0)
    except pd.errors.ParserError:
        # If ParserError, try reading with header=None and inferring header
        df = pd.read_csv(file_path, header=None)
        df.columns = df.iloc[0]  # Set first row as header
        df = df[1:]  # Drop the original first row (duplicate header)
        df = df.reset_index(drop=True)  # Reset index

    return df

def load_and_clean_excel(file_path):
    """Loads an Excel file, handling potential extra header rows."""

    # Attempt to read with header=0 (default)
    try:
        df = pd.read_excel(file_path, header=0)
    except pd.errors.ParserError:
        # If ParserError, try reading with header=None and inferring header
        df = pd.read_excel(file_path, header=None)
        df.columns = df.iloc[0]  # Set first row as header
        df = df[1:]  # Drop the original first row (duplicate header)
        df = df.reset_index(drop=True)  # Reset index

    return df

# Diesel data cleanup process
diesel_df = pd.read_excel("/content/diesel-240.xlsx", skiprows=1)
diesel_df = diesel_df.rename(columns={'Month': 'Date'})

# Rainfall data cleanup
rainfall_df = load_and_clean_csv("/content/Rainfall Sub_Division.csv")

# CPI data cleanup
cpi_df = load_and_clean_excel("/content/CPI.xlsx")

# MSP File: Delete unwanted rows in the header and keep the main header values from row number 4

# Load the MSP data, skipping specified rows and using row 4 as header
msp_df = pd.read_excel("/content/MSP.xlsx", header=None, skiprows=[0, 1, 2, 4]) # Skip rows 1, 2, 3, and 5 (0-based indexing)

# Set the 0th row (original row 4) as the header
msp_df.columns = msp_df.iloc[0]

# Drop the 0th row (duplicate header)
msp_df = msp_df[1:]

# Reset the index
msp_df = msp_df.reset_index(drop=True)

# Drop the first column (index 0)
msp_df = msp_df.drop(columns=msp_df.columns[0])

# Display the cleaned data structures
print("Diesel Dataset Structure:")
print(diesel_df.columns)
print(diesel_df.shape)  # Print (rows, columns)

print("\nRainfall Dataset Structure:")
print(rainfall_df.columns)
print(rainfall_df.shape)

print("\nCPI Dataset Structure:")
print(cpi_df.columns)
print(cpi_df.shape)

print("\nMSP Dataset Structure:")
print(msp_df.columns)
print(msp_df.shape)  # Print (rows, columns)

Diesel Dataset Structure:
Index(['Date', 'Diesel Price (Indian Rupee per Gallon)',
       'Wheat Price (Indian Rupee per Metric Ton)', 'Diesel ROC', 'Wheat ROC',
       'Diesel / Wheat Price Ratio'],
      dtype='object')
(222, 6)

Rainfall Dataset Structure:
Index(['SUBDIVISION', 'YEAR', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL',
       'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'ANNUAL', 'JF', 'MAM', 'JJAS',
       'OND'],
      dtype='object')
(4188, 19)

CPI Dataset Structure:
Index(['Year', 'CPI'], dtype='object')
(64, 2)

MSP Dataset Structure:
Index(['Year', 'Paddy common', 'Maize', 'Wheat', 'Gram', 'Arhar (Tur)',
       'Moong', 'Urad'],
      dtype='object', name=0)
(34, 8)


In [79]:
# Convert date column in wheat dataset to datetime and extract year
wheat_df['date'] = pd.to_datetime(wheat_df['date'], errors='coerce')
wheat_df['year'] = wheat_df['date'].dt.year

# Check CPI date format
cpi_df.columns = cpi_df.columns.str.strip()
cpi_df['Year'] = pd.to_numeric(cpi_df['Year'], errors='coerce')

# Compare the unique years between the two datasets
unique_wheat_years = sorted(wheat_df['year'].dropna().unique())
unique_cpi_years = sorted(cpi_df['Year'].dropna().unique())

(unique_wheat_years, unique_cpi_years)


([np.int32(1994),
  np.int32(1995),
  np.int32(1996),
  np.int32(1997),
  np.int32(1998),
  np.int32(1999),
  np.int32(2000),
  np.int32(2001),
  np.int32(2002),
  np.int32(2003),
  np.int32(2004),
  np.int32(2005),
  np.int32(2006),
  np.int32(2007),
  np.int32(2008),
  np.int32(2009),
  np.int32(2010),
  np.int32(2011),
  np.int32(2012),
  np.int32(2013),
  np.int32(2014),
  np.int32(2015),
  np.int32(2016),
  np.int32(2017),
  np.int32(2018),
  np.int32(2019),
  np.int32(2020),
  np.int32(2021),
  np.int32(2022)],
 [np.int64(1960),
  np.int64(1961),
  np.int64(1962),
  np.int64(1963),
  np.int64(1964),
  np.int64(1965),
  np.int64(1966),
  np.int64(1967),
  np.int64(1968),
  np.int64(1969),
  np.int64(1970),
  np.int64(1971),
  np.int64(1972),
  np.int64(1973),
  np.int64(1974),
  np.int64(1975),
  np.int64(1976),
  np.int64(1977),
  np.int64(1978),
  np.int64(1979),
  np.int64(1980),
  np.int64(1981),
  np.int64(1982),
  np.int64(1983),
  np.int64(1984),
  np.int64(1985),
  np.int6

In [80]:
# Compare the unique years between the two datasets wheat and diesel

# Convert the 'Date' column in diesel_df to datetime objects
diesel_df['Date'] = pd.to_datetime(diesel_df['Date'], errors='coerce')

# Extract the year from the 'Date' column
diesel_df['year'] = diesel_df['Date'].dt.year

# Get unique years from the 'year' column in diesel_df
unique_diesel_years = sorted(diesel_df['year'].dropna().unique())

# Compare unique years between wheat and diesel datasets
print("Unique Wheat Years:", unique_wheat_years)
print("Unique Diesel Years:", unique_diesel_years)

# Find common years
common_years = set(unique_wheat_years) & set(unique_diesel_years)
print("Common Years:", sorted(list(common_years)))


Unique Wheat Years: [np.int32(1994), np.int32(1995), np.int32(1996), np.int32(1997), np.int32(1998), np.int32(1999), np.int32(2000), np.int32(2001), np.int32(2002), np.int32(2003), np.int32(2004), np.int32(2005), np.int32(2006), np.int32(2007), np.int32(2008), np.int32(2009), np.int32(2010), np.int32(2011), np.int32(2012), np.int32(2013), np.int32(2014), np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022)]
Unique Diesel Years: [np.int32(2006), np.int32(2007), np.int32(2008), np.int32(2009), np.int32(2010), np.int32(2011), np.int32(2012), np.int32(2013), np.int32(2014), np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023), np.int32(2024)]
Common Years: [np.int32(2006), np.int32(2007), np.int32(2008), np.int32(2009), np.int32(2010), np.int32(2011), np.int32(2012), np.int32(2013), np.int32(2014), np.int32(2015), np.int32(2016)

Observed multiple roles of Catgory for each product and each state that needs to be grouped and average out the price as State.

In [81]:
# Drop specified columns from wheat_df
wheat_df = wheat_df.drop(columns=['region', 'market', 'unit', 'priceflag','latitude','longitude','usdprice'])
wheat_df.columns


Index(['date', 'state', 'market_id', 'category', 'commodity', 'commodity_id',
       'pricetype', 'currency', 'price', 'year'],
      dtype='object')

In [82]:
# Convert 'date' to datetime format (optional but helps with sorting/filtering)
wheat_df['date'] = pd.to_datetime(wheat_df['date'])

# Group by the required columns and calculate average price
# Convert 'price' column to numeric, handling errors by coercing to NaN
wheat_df['price'] = pd.to_numeric(wheat_df['price'], errors='coerce')
wheat_grouped_df = wheat_df.groupby(
    ['date', 'state', 'category', 'commodity', 'pricetype', 'currency', 'year'],
    as_index=False
)['price'].mean()

# View the final DataFrame
print(wheat_grouped_df.head())

        date           state            category commodity pricetype currency  \
0 1994-01-15           Delhi  cereals and tubers     Wheat    Retail      INR   
1 1994-01-15       Karnataka  cereals and tubers     Wheat    Retail      INR   
2 1994-01-15  Madhya Pradesh  cereals and tubers     Wheat    Retail      INR   
3 1994-02-15           Delhi  cereals and tubers     Wheat    Retail      INR   
4 1994-02-15         Gujarat  cereals and tubers     Wheat    Retail      INR   

   year  price  
0  1994    5.0  
1  1994    7.0  
2  1994    4.5  
3  1994    5.2  
4  1994    6.0  


Verifying if the grouping was correct for any given state, year 2020 jan,

In [83]:
# Filter for Bihar in 2020
up_2020_df = wheat_grouped_df[(wheat_grouped_df['state'] == 'Bihar') & (wheat_grouped_df['year'] == 2020)]

# Filter for January 2020
up_jan_2020_df = up_2020_df[up_2020_df['date'].dt.month == 1]

up_jan_2020_df


,date,state,category,commodity,pricetype,currency,year,price
4343,2020-01-15,Bihar,cereals and tubers,Wheat,Retail,INR,2020,22.0
4344,2020-01-15,Bihar,cereals and tubers,Wheat,Wholesale,INR,2020,1850.0


In [84]:
wheat_grouped_df

,date,state,category,commodity,pricetype,currency,year,price
0,1994-01-15,Delhi,cereals and tubers,Wheat,Retail,INR,1994,5.000000
1,1994-01-15,Karnataka,cereals and tubers,Wheat,Retail,INR,1994,7.000000
2,1994-01-15,Madhya Pradesh,cereals and tubers,Wheat,Retail,INR,1994,4.500000
3,1994-02-15,Delhi,cereals and tubers,Wheat,Retail,INR,1994,5.200000
4,1994-02-15,Gujarat,cereals and tubers,Wheat,Retail,INR,1994,6.000000
...,...,...,...,...,...,...,...,...
5233,2022-12-15,Tamil Nadu,cereals and tubers,Wheat,Retail,INR,2022,39.508000
5234,2022-12-15,Telangana,cereals and tubers,Wheat,Retail,INR,2022,35.378000
5235,2022-12-15,Uttar Pradesh,cereals and tubers,Wheat,Retail,INR,2022,29.348571
5236,2022-12-15,Uttarakhand,cereals and tubers,Wheat,Retail,INR,2022,27.973333


In [85]:
# check count of rows in wheat grouped df

print(len(wheat_grouped_df))


5238


In [86]:
### Step 5: Merging CPI values to Wheat dataset

In [87]:
# Parse wheat date and extract year
wheat_grouped_df['date'] = pd.to_datetime(wheat_grouped_df['date'], errors='coerce')
wheat_grouped_df['year'] = wheat_grouped_df['date'].dt.year

# Clean CPI column names (if needed)
cpi_df.columns = cpi_df.columns.str.strip()  # remove extra spaces
cpi_df = cpi_df.rename(columns={"Year": "year"})  # align column name for merge

# Merge CPI into wheat data on 'year'
wheat_with_cpi = pd.merge(wheat_grouped_df, cpi_df, on='year', how='left')
wheat_merge1 = wheat_with_cpi.copy()

# Quick check
print(wheat_merge1.head(10))
print("Count of records in the merged wheat dataset: ", len(wheat_merge1)) # keep track fo rows in main dataset post merging ( left join)


        date           state            category commodity pricetype currency  \
0 1994-01-15           Delhi  cereals and tubers     Wheat    Retail      INR   
1 1994-01-15       Karnataka  cereals and tubers     Wheat    Retail      INR   
2 1994-01-15  Madhya Pradesh  cereals and tubers     Wheat    Retail      INR   
3 1994-02-15           Delhi  cereals and tubers     Wheat    Retail      INR   
4 1994-02-15         Gujarat  cereals and tubers     Wheat    Retail      INR   
5 1994-02-15  Madhya Pradesh  cereals and tubers     Wheat    Retail      INR   
6 1994-02-15     Maharashtra  cereals and tubers     Wheat    Retail      INR   
7 1994-02-15      Tamil Nadu  cereals and tubers     Wheat    Retail      INR   
8 1994-02-15       Telangana  cereals and tubers     Wheat    Retail      INR   
9 1994-03-15         Gujarat  cereals and tubers     Wheat    Retail      INR   

   year  price    CPI  
0  1994    5.0  10.25  
1  1994    7.0  10.25  
2  1994    4.5  10.25  
3  1994    5

To check for top 10 2017 wheat process rows and corresponding CPI which shoudl be the same as the granularity is at year level when mapping was done

In [88]:

wheat_merge1[wheat_merge1['year'] == 2017][['date', 'state', 'price', 'CPI']].head(10)
# 'market' column has been removed from the list of columns being selected.


,date,state,price,CPI
3592,2017-01-15,Andaman and Nicobar,32.0000,3.33
3593,2017-01-15,Andhra Pradesh,27.4800,3.33
3594,2017-01-15,Assam,26.8600,3.33
3595,2017-01-15,Bihar,23.0000,3.33
3596,2017-01-15,Bihar,2113.3300,3.33
3597,2017-01-15,Chandigarh,22.0000,3.33
3598,2017-01-15,Delhi,22.7300,3.33
3599,2017-01-15,Delhi,2057.2700,3.33
3600,2017-01-15,Gujarat,23.1350,3.33
3601,2017-01-15,Haryana,22.7925,3.33


### Step 6: Merging MSP data to Wheat Dataset
The MSP value announced in October applies to the harvest from March of the same year until February of the following year.
#### Rule is:

*   MSP announced in October 2021 applies to wheat harvested from March 2022 to February 2023.
*   MSP announced in October 2022 applies to wheat harvested from March 2023 to February 2024.

#### Logic for Assigning MSP:


*   For March to February (regardless of the specific year):The MSP announced in October of the previous year applies.
*   For example, for the period March 2022 to February 2023, the MSP announced in October 2021 will apply.

Notes :
 * 1. Wheat prices for 1992-93 and 1993-94, include a Central bonus of ₹25 per quintal.'
 * 2. Wheat price for 2006-07 and Paddy common for year 2007-08 include an additional incentive bonus of ₹100 per quintal is payable  over the MSP.'
 * 3. Wheat prices for 1996-97 includes a Central bonus of ₹60 per quintal payable up to June 30, 1997.'
 * 4. Wheat prices for 2005-06 and 2010-11 include a bonus of ₹50 per quintal payable over the MSP.'


In [89]:
print(msp_df.columns.tolist())


['Year', 'Paddy common', 'Maize', 'Wheat', 'Gram', 'Arhar (Tur)', 'Moong', 'Urad']


In [90]:
# Ensure datetime format and extract year/month from wheat data
wheat_merge1['date'] = pd.to_datetime(wheat_merge1['date'], errors='coerce')
wheat_merge1['year'] = wheat_merge1['date'].dt.year
wheat_merge1['month'] = wheat_merge1['date'].dt.month

# Assign MSP year logic: Oct of previous year applies to Mar–Feb
wheat_merge1['msp_year'] = wheat_merge1.apply(
    lambda x: x['year'] - 1 if x['month'] <= 2 else x['year'], axis=1
)

# Clean and extract MSP year from msp_df
msp_df = msp_df.dropna(subset=['Year'])  # Remove NaNs in Year
msp_df['Year'] = msp_df['Year'].astype(str)

# Safely extract the 4-digit starting year even from ranges like '1991-92'
msp_df['MSP_Year'] = msp_df['Year'].apply(lambda x: int(x.split('-')[0]) if '-' in x else int(x))

# Merge: Keep all columns from both datasets
wheat_merge2 = pd.merge(
    wheat_merge1,
    msp_df,
    how='left',
    left_on='msp_year',
    right_on='MSP_Year'
)
# Rename the 'Wheat' column to 'MSP_Wheat' in wheat_merge2
wheat_merge2 = wheat_merge2.rename(columns={'Wheat': 'MSP_Wheat'})

# Preview merged data
wheat_merge2.head()

,date,state,category,commodity,pricetype,currency,year,price,CPI,month,msp_year,Year,Paddy common,Maize,MSP_Wheat,Gram,Arhar (Tur),Moong,Urad,MSP_Year
0,1994-01-15,Delhi,cereals and tubers,Wheat,Retail,INR,1994,5.0,10.25,1,1993,1993-94,310,260,350,640,700,700,700,1993
1,1994-01-15,Karnataka,cereals and tubers,Wheat,Retail,INR,1994,7.0,10.25,1,1993,1993-94,310,260,350,640,700,700,700,1993
2,1994-01-15,Madhya Pradesh,cereals and tubers,Wheat,Retail,INR,1994,4.5,10.25,1,1993,1993-94,310,260,350,640,700,700,700,1993
3,1994-02-15,Delhi,cereals and tubers,Wheat,Retail,INR,1994,5.2,10.25,2,1993,1993-94,310,260,350,640,700,700,700,1993
4,1994-02-15,Gujarat,cereals and tubers,Wheat,Retail,INR,1994,6.0,10.25,2,1993,1993-94,310,260,350,640,700,700,700,1993


In [91]:
# find unique values from wheat msp column in wheat merge2 dataframe
unique_msp_values = wheat_merge2['MSP_Wheat'].unique()
unique_msp_values


array([350, 360, 380, 475, 510, 550, 580, 610, 620, 630, 640, 650, 850,
       1000, 1080, 1100, 1170, 1285, 1350, 1400, 1450, 1525, 1625, 1735,
       1840, 1925, 1975, 2015, 2125], dtype=object)

Dropping off the non -Wheat food product columns from the new data frame

In [92]:
# Columns to drop
columns_to_drop = ['Paddy common', 'Maize', 'Gram', 'Arhar (Tur)', 'Moong', 'Urad']

# Drop the columns
wheat_merge2 = wheat_merge2.drop(columns=columns_to_drop, errors='ignore')

# Display the updated DataFrame (optional)
print(wheat_merge2.head())

        date           state            category commodity pricetype currency  \
0 1994-01-15           Delhi  cereals and tubers     Wheat    Retail      INR   
1 1994-01-15       Karnataka  cereals and tubers     Wheat    Retail      INR   
2 1994-01-15  Madhya Pradesh  cereals and tubers     Wheat    Retail      INR   
3 1994-02-15           Delhi  cereals and tubers     Wheat    Retail      INR   
4 1994-02-15         Gujarat  cereals and tubers     Wheat    Retail      INR   

   year  price    CPI  month  msp_year     Year MSP_Wheat  MSP_Year  
0  1994    5.0  10.25      1      1993  1993-94       350      1993  
1  1994    7.0  10.25      1      1993  1993-94       350      1993  
2  1994    4.5  10.25      1      1993  1993-94       350      1993  
3  1994    5.2  10.25      2      1993  1993-94       350      1993  
4  1994    6.0  10.25      2      1993  1993-94       350      1993  


In [93]:
# Check for redundancy between 'msp_year' and 'MSP_YEAR'
if 'MSP_Year' in wheat_merge2.columns and 'msp_year' in wheat_merge2.columns:
    # Compare the columns for equality
    if wheat_merge2['MSP_Year'].equals(wheat_merge2['msp_year']):
        # Drop 'MSP_Year' if they are identical
        wheat_merge2 = wheat_merge2.drop(columns=['MSP_Year'])
        print("'MSP_Year' column dropped as it's redundant.")
    else:
        print("'msp_year' and 'MSP_Year' columns are not identical.")
elif 'MSP_Year' not in wheat_merge2.columns:
    print("'MSP_Year' column not found in DataFrame.")
elif 'msp_year' not in wheat_merge2.columns:
    print("'msp_year' column not found in DataFrame.")


'MSP_Year' column dropped as it's redundant.


### Step 7: Merging Diesel Price data to Wheat Dataset

In [94]:
# Convert 'Date' and 'date' columns to datetime format
diesel_df['Date'] = pd.to_datetime(diesel_df['Date'])
wheat_merge2['date'] = pd.to_datetime(wheat_merge2['date'])

# Create a 'year_month' column in both dataframes
diesel_df['year_month'] = diesel_df['Date'].dt.to_period('M')
wheat_merge2['year_month'] = wheat_merge2['date'].dt.to_period('M')

# Merge the two DataFrames on 'year_month'
wheat_merge3 = pd.merge(wheat_merge2, diesel_df, how='left', on='year_month')

#Rename the column  Diesel Price (Indian Rupee per Gallon)  to Diesel Price in merge 3 dataframe
wheat_merge3 = wheat_merge3.rename(columns={'Diesel Price (Indian Rupee per Gallon)': 'Diesel Price'})

# Print the merged DataFrame
print(wheat_merge3.head())

        date           state            category commodity pricetype currency  \
0 1994-01-15           Delhi  cereals and tubers     Wheat    Retail      INR   
1 1994-01-15       Karnataka  cereals and tubers     Wheat    Retail      INR   
2 1994-01-15  Madhya Pradesh  cereals and tubers     Wheat    Retail      INR   
3 1994-02-15           Delhi  cereals and tubers     Wheat    Retail      INR   
4 1994-02-15         Gujarat  cereals and tubers     Wheat    Retail      INR   

   year_x  price    CPI  month  ...     Year MSP_Wheat year_month Date  \
0    1994    5.0  10.25      1  ...  1993-94       350    1994-01  NaT   
1    1994    7.0  10.25      1  ...  1993-94       350    1994-01  NaT   
2    1994    4.5  10.25      1  ...  1993-94       350    1994-01  NaT   
3    1994    5.2  10.25      2  ...  1993-94       350    1994-02  NaT   
4    1994    6.0  10.25      2  ...  1993-94       350    1994-02  NaT   

  Diesel Price  Wheat Price (Indian Rupee per Metric Ton)  Diesel RO

Check whether there are non NAN values for disel price column

In [95]:
print(wheat_merge3['Diesel ROC'].unique())

[nan '-' 0.0695 0.0153 -0.2017 -0.0229 -0.0041 0.0071 -0.1166 0.0748
 0.0717 0.0394 -0.0365 0.0364 0.0037 -0.0089 0.0887 0.0205 0.115 -0.0256
 -0.0173 0.0807 0.1912 0.0499 0.1679 0.034 -0.0104 -0.1467 -0.0141 -0.1585
 -0.1918 -0.2588 0.046 -0.1202 0.0671 0.0576 0.0168 0.1786 -0.0393 0.1147
 -0.0662 0.065 0.0048 -0.0077 0.0277 -0.0064 0.042 0.0347 -0.0381 0.004
 -0.0135 0.0204 0.0133 0.0207 0.0437 0.0586 0.0618 0.0766 0.0912 0.0307
 -0.0615 0.0053 0.0279 -0.032 0.0436 0.0414 0.0592 -0.0145 0.0043 0.0487
 0.0107 -0.0336 -0.0645 0.0682 0.089 -0.0026 -0.0272 0.0154 -0.0364 0.0051
 0.0322 -0.0505 -0.0453 0.0079 0.061 0.0699 0.0749 0.001 -0.0491 -0.0019
 0.0238 0.0267 0.0378 -0.0921 -0.0216 -0.0241 0.0151 -0.0223 0.0036
 -0.0388 -0.0689 -0.0351 -0.1489 -0.1826 0.1845 -0.0022 0.0842 -0.0384
 -0.1111 -0.0796 0.0176 -0.0457 -0.0257 -0.188 -0.1307 0.0732 0.1287
 0.0387 0.1483 0.054 -0.082 0.0169 0.0106 0.0997 -0.0508 0.1285 -0.0085
 -0.0758 0.0159 -0.0468 -0.0638 0.0735 0.0644 0.1066 0.0142 0.06

### Step 8: Formatting and melting the raw Rainfall data frame and reshaping

In [96]:
# Displaying Existing data
rainfall_df.head()

,SUBDIVISION,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL,JF,MAM,JJAS,OND
0,Andaman & Nicobar Islands,1901,49.2,87.1,29.2,2.3,528.8,517.5,365.1,481.1,332.6,388.5,558.2,33.6,3373.2,136.3,560.3,1696.3,980.3
1,Andaman & Nicobar Islands,1902,0.0,159.8,12.2,0.0,446.1,537.1,228.9,753.7,666.2,197.2,359.0,160.5,3520.7,159.8,458.3,2185.9,716.7
2,Andaman & Nicobar Islands,1903,12.7,144.0,0.0,1.0,235.1,479.9,728.4,326.7,339.0,181.2,284.4,225.0,2957.4,156.7,236.1,1874.0,690.6
3,Andaman & Nicobar Islands,1904,9.4,14.7,0.0,202.4,304.5,495.1,502.0,160.1,820.4,222.2,308.7,40.1,3079.6,24.1,506.9,1977.6,571.0
4,Andaman & Nicobar Islands,1905,1.3,0.0,3.3,26.9,279.5,628.7,368.7,330.5,297.0,260.7,25.4,344.7,2566.7,1.3,309.7,1624.9,630.8


In [97]:
# Melt the DataFrame to create 'Month' and 'Rainfall' columns
rainfall_melted = pd.melt(
    rainfall_df,
    id_vars=['SUBDIVISION', 'YEAR'],
    value_vars=[
        'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
        'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'
    ],
    var_name='Month',
    value_name='Rainfall'
)

# Merge the melted DataFrame with the original DataFrame to add aggregated columns
rainfall_reshaped = pd.merge(
    rainfall_melted,
    rainfall_df[['SUBDIVISION', 'YEAR', 'ANNUAL', 'JF', 'MAM', 'JJAS', 'OND']],
    on=['SUBDIVISION', 'YEAR'],
    how='left'
)

# Display the reshaped DataFrame
print(rainfall_reshaped.head())

                 SUBDIVISION  YEAR Month  Rainfall  ANNUAL     JF    MAM  \
0  Andaman & Nicobar Islands  1901   JAN      49.2  3373.2  136.3  560.3   
1  Andaman & Nicobar Islands  1902   JAN       0.0  3520.7  159.8  458.3   
2  Andaman & Nicobar Islands  1903   JAN      12.7  2957.4  156.7  236.1   
3  Andaman & Nicobar Islands  1904   JAN       9.4  3079.6   24.1  506.9   
4  Andaman & Nicobar Islands  1905   JAN       1.3  2566.7    1.3  309.7   

     JJAS    OND  
0  1696.3  980.3  
1  2185.9  716.7  
2  1874.0  690.6  
3  1977.6  571.0  
4  1624.9  630.8  


Regions and subdivsion not common in both data frames wheat price and Rainfall, as the granularity has to be by the state

In [98]:
# Extract unique values from relevant columns
wheat_states = wheat_merge3['state'].unique()
rainfall_subdivisions = rainfall_reshaped['SUBDIVISION'].unique()

# Check for common values and potential issues
common_states = set(wheat_states) & set(rainfall_subdivisions)
wheat_only_states = set(wheat_states) - set(rainfall_subdivisions)
rainfall_only_subdivisions = set(rainfall_subdivisions) - set(wheat_states)

# Print results
print("Common States/Subdivisions:", common_states)
print("\nStates in Wheat Data but not in Rainfall Data:", wheat_only_states)
print("\nSubdivisions in Rainfall Data but not in Wheat Data:", rainfall_only_subdivisions)

# Check for case sensitivity issues, handling potential float values
wheat_states_lower = [str(state).lower() for state in wheat_merge3['state']]
rainfall_subdivisions_lower = [str(subdivision).lower() for subdivision in rainfall_subdivisions]
common_states_lower = set(wheat_states_lower) & set(rainfall_subdivisions_lower)

if len(common_states) < len(common_states_lower):
    print("\nWarning: Potential case sensitivity issues detected. Consider converting to lowercase before mapping.")



Common States/Subdivisions: {'Orissa', 'Himachal Pradesh', 'Uttarakhand', 'Chhattisgarh', 'Telangana', 'Kerala', 'Tamil Nadu', 'Punjab', 'Bihar', 'Jharkhand'}

States in Wheat Data but not in Rainfall Data: {'Uttar Pradesh', 'Karnataka', 'Assam', 'West Bengal', 'Madhya Pradesh', 'Puducherry', 'Mizoram', 'Chandigarh', 'Tripura', 'Delhi', 'Andaman and Nicobar', 'Meghalaya', 'Andhra Pradesh', 'Rajasthan', 'Gujarat', 'Haryana', 'Sikkim', 'Maharashtra', 'Nagaland', 'Goa', 'Manipur'}

Subdivisions in Rainfall Data but not in Wheat Data: {'Lakshadweep', 'East Madhya Pradesh', 'Vidarbha', 'Konkan & Goa', 'Haryana Delhi & Chandigarh', 'North Interior Karnataka', 'Coastal Karnataka', 'Jammu & Kashmir', 'East Rajasthan', 'Rayalseema', 'West Madhya Pradesh', 'East Uttar Pradesh', 'South Interior Karnataka', 'Naga Mani Mizo Tripura', 'West Uttar Pradesh', 'Gangetic West Bengal', 'Arunachal Pradesh', 'Assam & Meghalaya', 'Coastal Andhra Pradesh', 'Saurashtra & Kutch', 'Matathwada', 'Gujarat Region',

Identifying Common states from both the wheat and Rainfall datasets

In [99]:
# Extract unique values from relevant columns, converting to lowercase
wheat_states = wheat_merge3['state'].unique()  # Use 'state' column
rainfall_subdivisions = rainfall_reshaped['SUBDIVISION'].unique()

# Check for case sensitivity issues, handling potential float values
wheat_states_lower = [str(state).lower() for state in wheat_states]
rainfall_subdivisions_lower = [str(subdivision).lower() for subdivision in rainfall_subdivisions]

# Find common regions (case-insensitive)
common_states_lower = set(wheat_states_lower) & set(rainfall_subdivisions_lower)

# Print the common regions
print("Common States/Subdivisions (case-insensitive):", common_states_lower)

Common States/Subdivisions (case-insensitive): {'uttarakhand', 'tamil nadu', 'kerala', 'punjab', 'himachal pradesh', 'chhattisgarh', 'orissa', 'telangana', 'jharkhand', 'bihar'}


The rainfall_reshaped DataFrame needs to be modified to align with the desired state names and aggregations for certain regions. This involves renaming, splitting, and averaging values based on the provided instructions.

**CHANGES:**

Rainfall Dataset	Changes
* Coastal Karnataka, North Interior Karnataka,South Interior Karnataka	Take average of these three and map to Karnataka
* Andaman & Nicobar Islands	Change name to Andaman and Nicobar
* Assam & Meghalaya	Split into Assam and Meghalaya as the sub division name
* Konkan & Goa	Change subdivision name to Goa
* Haryana Delhi & Chandigarh	Split into Haryana Delhi and Chandigarh
* Gujarat Region	change name to Gujarat
* East Madhya Pradesh,West Madhya Pradesh,Madhya Maharashtra	Average these three and rename subdivision to Madhya Pradesh
* Naga Mani Mizo Tripura	Split into Nagaland, Manipur, Mizoram and Tripura with same row values fo rrest of the columns
* West Rajasthan,East Rajasthan	AVERAGE THESE TWO AND rename to Rajasthan as single row
* Sub Himalayan West Bengal & Sikkim	Split into Sikkim, and carry the values, also  take the values from here average with   Gangetic West Bengal there by change the name to West Bengal
* East Uttar Pradesh,West Uttar Pradesh	Average these two and make a single row of values with subdivision name as Uttar Pradesh
* Coastal Andhra Pradesh	change name to Andhra Pradesh


In [100]:
# Function to apply the transformations
def transform_subdivision_names(df):
    # Coastal Karnataka - Average these values
    coastal_karnataka = df[df["SUBDIVISION"].isin(["Coastal Karnataka", "North Interior Karnataka", "South Interior Karnataka"])]
    coastal_karnataka_avg = coastal_karnataka.mean(numeric_only=True).reset_index()
    coastal_karnataka_avg["SUBDIVISION"] = "Karnataka"
    df = df[~df["SUBDIVISION"].isin(["Coastal Karnataka", "North Interior Karnataka", "South Interior Karnataka"])]
    df = pd.concat([df, coastal_karnataka_avg])

    # Rename Andaman & Nicobar Islands to Andaman and Nicobar
    df["SUBDIVISION"] = df["SUBDIVISION"].replace("Andaman & Nicobar Islands", "Andaman and Nicobar")

    # Split Assam & Meghalaya
    assam_meghalaya = df[df["SUBDIVISION"] == "Assam & Meghalaya"]
    assam = assam_meghalaya.copy()
    assam["SUBDIVISION"] = "Assam"
    meghalaya = assam_meghalaya.copy()
    meghalaya["SUBDIVISION"] = "Meghalaya"
    df = df[df["SUBDIVISION"] != "Assam & Meghalaya"]
    df = pd.concat([df, assam, meghalaya])

    # Rename Konkan & Goa to Goa
    df["SUBDIVISION"] = df["SUBDIVISION"].replace("Konkan & Goa", "Goa")

    # Split Haryana Delhi & Chandigarh
    haryana_delhi_chandigarh = df[df["SUBDIVISION"] == "Haryana Delhi & Chandigarh"]
    haryana = haryana_delhi_chandigarh.copy()
    haryana["SUBDIVISION"] = "Haryana"
    delhi = haryana_delhi_chandigarh.copy()
    delhi["SUBDIVISION"] = "Delhi"
    chandigarh = haryana_delhi_chandigarh.copy()
    chandigarh["SUBDIVISION"] = "Chandigarh"
    df = df[df["SUBDIVISION"] != "Haryana Delhi & Chandigarh"]
    df = pd.concat([df, haryana, delhi, chandigarh])

    # Rename Gujarat Region to Gujarat
    df["SUBDIVISION"] = df["SUBDIVISION"].replace("Gujarat Region", "Gujarat")

    # Average East and West Madhya Pradesh and rename to Madhya Pradesh
    east_west_madhya_pradesh = df[df["SUBDIVISION"].isin(["East Madhya Pradesh", "West Madhya Pradesh"])]
    madhya_pradesh_avg = east_west_madhya_pradesh.mean(numeric_only=True).reset_index()
    madhya_pradesh_avg["SUBDIVISION"] = "Madhya Pradesh"
    df = df[~df["SUBDIVISION"].isin(["East Madhya Pradesh", "West Madhya Pradesh"])]
    df = pd.concat([df, madhya_pradesh_avg])

    # Rename Madhya Maharashtra to Maharashtra
    df["SUBDIVISION"] = df["SUBDIVISION"].replace("Madhya Maharashtra", "Maharashtra")

    # Split Naga Mani Mizo Tripura into individual states
    nmt = df[df["SUBDIVISION"] == "Naga Mani Mizo Tripura"]
    nagaland = nmt.copy()
    nagaland["SUBDIVISION"] = "Nagaland"
    manipur = nmt.copy()
    manipur["SUBDIVISION"] = "Manipur"
    mizoram = nmt.copy()
    mizoram["SUBDIVISION"] = "Mizoram"
    tripura = nmt.copy()
    tripura["SUBDIVISION"] = "Tripura"
    df = df[df["SUBDIVISION"] != "Naga Mani Mizo Tripura"]
    df = pd.concat([df, nagaland, manipur, mizoram, tripura])

    # Average West Rajasthan and East Rajasthan and rename to Rajasthan
    west_east_rajasthan = df[df["SUBDIVISION"].isin(["West Rajasthan", "East Rajasthan"])]
    rajasthan_avg = west_east_rajasthan.mean(numeric_only=True).reset_index()
    rajasthan_avg["SUBDIVISION"] = "Rajasthan"
    df = df[~df["SUBDIVISION"].isin(["West Rajasthan", "East Rajasthan"])]
    df = pd.concat([df, rajasthan_avg])

    # Rename Sub Himalayan West Bengal & Sikkim to Sikkim
    df["SUBDIVISION"] = df["SUBDIVISION"].replace("Sub Himalayan West Bengal & Sikkim", "Sikkim")

    # Average East and West Uttar Pradesh and rename to Uttar Pradesh
    east_west_up = df[df["SUBDIVISION"].isin(["East Uttar Pradesh", "West Uttar Pradesh"])]
    up_avg = east_west_up.mean(numeric_only=True).reset_index()
    up_avg["SUBDIVISION"] = "Uttar Pradesh"
    df = df[~df["SUBDIVISION"].isin(["East Uttar Pradesh", "West Uttar Pradesh"])]
    df = pd.concat([df, up_avg])

    # Rename Coastal Andhra Pradesh to Andhra Pradesh
    df["SUBDIVISION"] = df["SUBDIVISION"].replace("Coastal Andhra Pradesh", "Andhra Pradesh")

    # Rename Gangetic West Bengal to West Bengal
    df["SUBDIVISION"] = df["SUBDIVISION"].replace("Gangetic West Bengal", "West Bengal")

    return df

# Apply the transformations
rainfall_reshaped = transform_subdivision_names(rainfall_reshaped)

# Print the transformed DataFrame
print(rainfall_reshaped.head())

           SUBDIVISION    YEAR Month  Rainfall  ANNUAL     JF    MAM    JJAS  \
0  Andaman and Nicobar  1901.0   JAN      49.2  3373.2  136.3  560.3  1696.3   
1  Andaman and Nicobar  1902.0   JAN       0.0  3520.7  159.8  458.3  2185.9   
2  Andaman and Nicobar  1903.0   JAN      12.7  2957.4  156.7  236.1  1874.0   
3  Andaman and Nicobar  1904.0   JAN       9.4  3079.6   24.1  506.9  1977.6   
4  Andaman and Nicobar  1905.0   JAN       1.3  2566.7    1.3  309.7  1624.9   

     OND index   0  
0  980.3   NaN NaN  
1  716.7   NaN NaN  
2  690.6   NaN NaN  
3  571.0   NaN NaN  
4  630.8   NaN NaN  


In [101]:
# unique values of subdivision
unique_subdivisions = rainfall_reshaped['SUBDIVISION'].unique()
unique_subdivisions


array(['Andaman and Nicobar', 'Arunachal Pradesh', 'Sikkim',
       'West Bengal', 'Orissa', 'Jharkhand', 'Bihar', 'Uttarakhand',
       'Punjab', 'Himachal Pradesh', 'Jammu & Kashmir', 'Gujarat',
       'Saurashtra & Kutch', 'Goa', 'Maharashtra', 'Matathwada',
       'Vidarbha', 'Chhattisgarh', 'Andhra Pradesh', 'Telangana',
       'Rayalseema', 'Tamil Nadu', 'Kerala', 'Lakshadweep', 'Karnataka',
       'Assam', 'Meghalaya', 'Haryana', 'Delhi', 'Chandigarh',
       'Madhya Pradesh', 'Nagaland', 'Manipur', 'Mizoram', 'Tripura',
       'Rajasthan', 'Uttar Pradesh'], dtype=object)

In [102]:
#  Unique values of subdivision and corresponding rainfall values for any 2010 year all months

# Filter for the year 2010
rainfall_2010 = rainfall_reshaped[rainfall_reshaped['YEAR'] == 2010]

# Get unique subdivision names and their corresponding rainfall values
unique_subdivisions_2010 = rainfall_2010[['SUBDIVISION', 'Rainfall']].drop_duplicates()

unique_subdivisions_2010


,SUBDIVISION,Rainfall
104,Andaman and Nicobar,101.7
203,Arunachal Pradesh,0.6
554,Sikkim,5.6
671,West Bengal,0.2
788,Orissa,8.8
...,...,...
29753,Tripura,393.2
33941,Tripura,309.8
38129,Tripura,220.2
42317,Tripura,9.2


In [103]:
# Showing the complete line items for sikkim as state

# Filter for Sikkim
sikkim_rainfall = rainfall_reshaped[rainfall_reshaped['SUBDIVISION'] == 'Sikkim']

# Display the complete line items for Sikkim
sikkim_rainfall


,SUBDIVISION,YEAR,Month,Rainfall,ANNUAL,JF,MAM,JJAS,OND,index,0
445,Sikkim,1901.0,JAN,26.5,2113.2,41.3,238.9,1757.0,76.1,NaN,NaN
446,Sikkim,1902.0,JAN,1.2,3180.4,1.9,484.6,2613.9,80.1,NaN,NaN
447,Sikkim,1903.0,JAN,5.5,2404.5,14.3,201.9,2046.7,141.6,NaN,NaN
448,Sikkim,1904.0,JAN,3.4,2169.9,32.5,458.8,1503.7,174.8,NaN,NaN
449,Sikkim,1905.0,JAN,12.0,3005.0,43.2,447.0,2405.6,109.3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
46625,Sikkim,2013.0,DEC,2.7,2406.1,26.7,443.4,1717.6,218.5,NaN,NaN
46626,Sikkim,2014.0,DEC,2.4,2322.6,26.9,394.2,1862.6,38.9,NaN,NaN
46627,Sikkim,2015.0,DEC,9.0,2518.6,30.7,518.5,1883.0,86.4,NaN,NaN
46628,Sikkim,2016.0,DEC,0.3,2624.8,27.9,434.0,2007.1,155.8,NaN,NaN


### Step 9: Merging Rainfall data to the Wheat Price data

In [104]:
wheat_merge3

,date,state,category,commodity,pricetype,currency,year_x,price,CPI,month,...,Year,MSP_Wheat,year_month,Date,Diesel Price,Wheat Price (Indian Rupee per Metric Ton),Diesel ROC,Wheat ROC,Diesel / Wheat Price Ratio,year_y
0,1994-01-15,Delhi,cereals and tubers,Wheat,Retail,INR,1994,5.000000,10.25,1,...,1993-94,350,1994-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,1994-01-15,Karnataka,cereals and tubers,Wheat,Retail,INR,1994,7.000000,10.25,1,...,1993-94,350,1994-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,1994-01-15,Madhya Pradesh,cereals and tubers,Wheat,Retail,INR,1994,4.500000,10.25,1,...,1993-94,350,1994-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,1994-02-15,Delhi,cereals and tubers,Wheat,Retail,INR,1994,5.200000,10.25,2,...,1993-94,350,1994-02,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,1994-02-15,Gujarat,cereals and tubers,Wheat,Retail,INR,1994,6.000000,10.25,2,...,1993-94,350,1994-02,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5233,2022-12-15,Tamil Nadu,cereals and tubers,Wheat,Retail,INR,2022,39.508000,6.70,12,...,2022-23,2125,2022-12,2022-12-01,256.72,31818.65,-0.2278,-0.08,0.0081,2022.0
5234,2022-12-15,Telangana,cereals and tubers,Wheat,Retail,INR,2022,35.378000,6.70,12,...,2022-23,2125,2022-12,2022-12-01,256.72,31818.65,-0.2278,-0.08,0.0081,2022.0
5235,2022-12-15,Uttar Pradesh,cereals and tubers,Wheat,Retail,INR,2022,29.348571,6.70,12,...,2022-23,2125,2022-12,2022-12-01,256.72,31818.65,-0.2278,-0.08,0.0081,2022.0
5236,2022-12-15,Uttarakhand,cereals and tubers,Wheat,Retail,INR,2022,27.973333,6.70,12,...,2022-23,2125,2022-12,2022-12-01,256.72,31818.65,-0.2278,-0.08,0.0081,2022.0


In [105]:
rainfall_reshaped.head()

,SUBDIVISION,YEAR,Month,Rainfall,ANNUAL,JF,MAM,JJAS,OND,index,0
0,Andaman and Nicobar,1901.0,JAN,49.2,3373.2,136.3,560.3,1696.3,980.3,NaN,NaN
1,Andaman and Nicobar,1902.0,JAN,0.0,3520.7,159.8,458.3,2185.9,716.7,NaN,NaN
2,Andaman and Nicobar,1903.0,JAN,12.7,2957.4,156.7,236.1,1874.0,690.6,NaN,NaN
3,Andaman and Nicobar,1904.0,JAN,9.4,3079.6,24.1,506.9,1977.6,571.0,NaN,NaN
4,Andaman and Nicobar,1905.0,JAN,1.3,2566.7,1.3,309.7,1624.9,630.8,NaN,NaN


In [106]:
# compare merge 3 and rainfall reshaped data frame on state name and subdivision and tell me what are not common names

# Extract unique values from relevant columns, converting to lowercase for comparison
wheat_states = wheat_merge3['state'].unique()
rainfall_subdivisions = rainfall_reshaped['SUBDIVISION'].unique()

wheat_states_lower = [str(state).lower() for state in wheat_states]
rainfall_subdivisions_lower = [str(subdivision).lower() for subdivision in rainfall_subdivisions]

# Find non-common names (case-insensitive)
wheat_only_states = set(wheat_states_lower) - set(rainfall_subdivisions_lower)
rainfall_only_subdivisions = set(rainfall_subdivisions_lower) - set(wheat_states_lower)


# Print results
print("\nStates in Wheat Data but not in Rainfall Data:", wheat_only_states)
print("\nSubdivisions in Rainfall Data but not in Wheat Data:", rainfall_only_subdivisions)



States in Wheat Data but not in Rainfall Data: {'puducherry'}

Subdivisions in Rainfall Data but not in Wheat Data: {'saurashtra & kutch', 'matathwada', 'vidarbha', 'rayalseema', 'arunachal pradesh', 'jammu & kashmir', 'lakshadweep'}


The ones that are not common in both have to be ignored as we do not have data on Rainfallfor them or doesnt have wheat price information

Merging Rainfall external data with Wheat Prices data

In [107]:

# Extract year and month from the 'date' column in wheatMSP_diesel_df
wheat_merge3['date_x'] = pd.to_datetime(wheat_merge3['date'], errors='coerce')
wheat_merge3['year'] = wheat_merge3['date'].dt.year
wheat_merge3['month'] = wheat_merge3['date'].dt.month

# Rename 'SUBDIVISION' to 'state' in rainfall_reshaped for merging
rainfall_reshaped = rainfall_reshaped.rename(columns={'SUBDIVISION': 'state'})

# Ensure 'Month' is lowercase and 'YEAR' is numeric in rainfall_reshaped
rainfall_reshaped['Month'] = rainfall_reshaped['Month'].str.lower()
rainfall_reshaped['YEAR'] = pd.to_numeric(rainfall_reshaped['YEAR'])

# Ensure 'month' columns have the same format before merging
# Create a dictionary to map numeric months to month abbreviations
month_map = {
    1: 'jan', 2: 'feb', 3: 'mar', 4: 'apr', 5: 'may', 6: 'jun',
    7: 'jul', 8: 'aug', 9: 'sep', 10: 'oct', 11: 'nov', 12: 'dec'
}

# Apply the mapping to the 'month' column in wheatMSP_diesel_df
wheat_merge3['month'] = wheat_merge3['month'].map(month_map)

# Merge the DataFrames

Wheat_Prices_merged_df = pd.merge(
    wheat_merge3,
    rainfall_reshaped,  # Use the entire rainfall_reshaped dataframe
    left_on=['state', 'year', 'month'],  # Use extracted year and month
    right_on=['state', 'YEAR', 'Month'],
    how='left'  # Use 'left' to keep all rows from wheat_merge3
)

# Drop duplicate columns if any
Wheat_Prices_merged_df = Wheat_Prices_merged_df.drop(columns=['YEAR', 'Month'], errors='ignore')

# Display the merged DataFrame
print(Wheat_Prices_merged_df.head())

        date           state            category commodity pricetype currency  \
0 1994-01-15           Delhi  cereals and tubers     Wheat    Retail      INR   
1 1994-01-15       Karnataka  cereals and tubers     Wheat    Retail      INR   
2 1994-01-15  Madhya Pradesh  cereals and tubers     Wheat    Retail      INR   
3 1994-02-15           Delhi  cereals and tubers     Wheat    Retail      INR   
4 1994-02-15         Gujarat  cereals and tubers     Wheat    Retail      INR   

   year_x  price    CPI month  ...     date_x  year Rainfall  ANNUAL    JF  \
0    1994    5.0  10.25   jan  ... 1994-01-15  1994     20.1   564.4  31.4   
1    1994    7.0  10.25   jan  ... 1994-01-15  1994      NaN     NaN   NaN   
2    1994    4.5  10.25   jan  ... 1994-01-15  1994      NaN     NaN   NaN   
3    1994    5.2  10.25   feb  ... 1994-02-15  1994     11.3   564.4  31.4   
4    1994    6.0  10.25   feb  ... 1994-02-15  1994      0.0  1534.5   7.2   

    MAM    JJAS  OND index   0  
0  16.7   5

In [108]:
Wheat_Prices_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5238 entries, 0 to 5237
Data columns (total 31 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   date                                       5238 non-null   datetime64[ns]
 1   state                                      5238 non-null   object        
 2   category                                   5238 non-null   object        
 3   commodity                                  5238 non-null   object        
 4   pricetype                                  5238 non-null   object        
 5   currency                                   5238 non-null   object        
 6   year_x                                     5238 non-null   int32         
 7   price                                      5238 non-null   float64       
 8   CPI                                        5238 non-null   float64       
 9   month              

Verification:  check for bihar state that has rainfall value for one random year and populate corresponding price column too


In [109]:
# Filter for Bihar state
bihar_data = Wheat_Prices_merged_df[Wheat_Prices_merged_df['state'] == 'bihar']

# Check if Bihar data exists
if not bihar_data.empty:
    # Get a list of unique years for Bihar
    bihar_years = bihar_data['year'].unique()

    # Select a random year
    if len(bihar_years) > 0:  # Check for empty list
        random_year = random.choice(bihar_years)
        print(f"Selected random year for Bihar: {random_year}")

        # Filter data for the selected year
        bihar_year_data = bihar_data[bihar_data['year'] == random_year]

        # Print the relevant information
        print("Rainfall and Price data for Bihar in", random_year, ":")
        print(bihar_year_data[['year', 'month', 'Rainfall', 'price']])  # Print the required columns
    else:
        print("No year data found for Bihar.")
else:
    print("No data found for Bihar.")


No data found for Bihar.


### Step 10: Save final merged wheat dataset in a csv file.

In [110]:
# Save the DataFrame to a CSV file
Wheat_Prices_merged_df.to_csv("wheat_prices_merged.csv", index=False)